In [16]:
import enum
from collections import defaultdict

import spacy
from tqdm import tqdm

In [97]:
import editdistance
import sys

rn = ['ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', 'xi', 'xii', 'xiii',
                 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', 'xxi', 'xxii']

rome_numbers = dict(zip(rn, range(2, 23)))

def numbers_from(s):
    res = set()
    for w in s.split():
        w = w.lower()
        if w.isdecimal():
            res.add(w)
        if w in rome_numbers:
            res.add(rome_numbers[w])
    return res                 
                 
                 
def is_number(s):
    return s.lower() in rome_numbers or s.isdecimal()
                     
def scaled_editdist(ans, cor):
    ans = ans.lower()
    cor = cor.lower()
    
    return editdistance.eval(ans, cor) / len(cor)
    
def single_match(a, c):
    numbers_c = numbers_from(c)
    numbers_a = numbers_from(a)
        
    return numbers_a == numbers_c and scaled_editdist(a, c) < 0.5
        
def match(ans, cor):
    return any(single_match(ans, c) for c in cor)

In [17]:
nlp = spacy.load("pl_core_news_md")

In [18]:
with open("/home/jdworzans/projects/chatbots/data/task2_questions_with_answers.tsv", "rt") as f:
    content = f.read()

In [20]:
questions_answers = []
for l in content.splitlines():
    question, *answers = l.split("\t")
    questions_answers.append((question, answers))

In [21]:
class QuestionType(enum.Enum):
    UNKNOWN = 0
    YES_NO = 1
    SELECT = 2
    NAME = 3

In [22]:
def determine_type(q: str):
    doc: spacy.tokens.Doc = nlp(q)
    last_sentence: spacy.tokens.Span = list(doc.sents)[-1]
    czy_count = 0
    for token in last_sentence:
        if token.lemma_ == "czy":
            czy_count += 1
    if czy_count > 0:
        if czy_count == 1 and last_sentence[0].lemma_ == "czy":
            return QuestionType.YES_NO
        else:
            return QuestionType.SELECT
    if q.startswith("Jak nazywa"):
        return QuestionType.NAME
    return QuestionType.UNKNOWN

In [23]:
qa_groups = defaultdict(list)

In [24]:
for q, answers in tqdm(questions_answers):
    group = determine_type(q)
    qa_groups[group].append((q, answers))

100%|██████████| 3500/3500 [00:35<00:00, 99.38it/s] 


In [26]:
for group, qa_group in qa_groups.items():
    print(group, len(qa_group))

QuestionType.NAME 616
QuestionType.UNKNOWN 2361
QuestionType.YES_NO 292
QuestionType.SELECT 231


# checks

In [27]:
for q, answers in qa_groups[QuestionType.UNKNOWN]:
    if any(answer.lower() in q.lower() for answer in answers):
        print(q, answers)

W zdaniu: „Szybko przeskoczył przez płot” który wyraz jest przysłówkiem? ['szybko']
Ile medali zdobyli Polacy na Mistrzostwach Świata w lekkiej atletyce w 2013 roku w Moskwie? ['trzy', '3']
Wskaż przyimek w zdaniu „Idę do szkoły”. ['do']
Które miasto jest stolicą Meksyku? ['Meksyk']
Ku czci której greckiej bogini organizowane były panatenaje? ['Atena']
Miłosz, Żeromski, Reymont – który z nich nie dostał Nagrody Nobla? ['Żeromski']
Który rozbiór Polski nastąpił w 1772 roku? ['pierwszy', '1']
Jaki pseudonim nosił Stanisław Wiechecki? ['Wiech']
Ile razy w ciągu życia kwitną agawy? ['raz', '1']
W filmie „Dyktator” Charlie Chaplin zagrał podwójną rolę. A kto był scenarzystą i reżyserem tego filmu? ['Charlie Chaplin']
A2, A3, A4 – który z tych arkuszy papieru ma najmniejsze wymiary? ['A4']
Którą wielką literą alfabetu oznacza się stałą grawitacji Newtona? ['G']
Jak w szachach nazywa się ruch powodujący bezpośrednie zagrożenie króla? ['szach']
Etna, Stromboli, Tambora – który wulkan nie wznos

In [29]:
for q, answers in qa_groups[QuestionType.UNKNOWN]:
    if any(answer.lower() in {"tak", "nie"} for answer in answers):
        print(q, answers)

Czy bulaj i iluminator są tym samym? ['nie']
1246 to liczba parzysta? ['tak']


In [34]:
for q, answers in qa_groups[QuestionType.YES_NO]:
    if all(answer.lower() not in {"tak", "nie"} for answer in answers):
        print(q, answers)

Czy choroba Heinego-Medina i dziecięce porażenie to ta sama choroba? ['wokół głowy']


In [35]:
for q, answers in qa_groups[QuestionType.SELECT]:
    if all(answer.lower() not in q.lower() for answer in answers):
        print(q, answers)

Wyspy Zielonego Przylądka leżą bliżej Afryki czy Azji? ['Kto śpiewa piosenkę z czołówki M jak miłość?']
Co ile lat odradzał się z popiołów mityczny feniks – co 50, co 500 czy co 5000 lat? ['co 500 lat']
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem']
Czy trybun ludowy był w starożytnym Rzymie urzędnikiem broniącym plebsu czy ministrem spraw zagranicznych? ['urzędnikiem broniącym spraw plebsu']
Kiedy powstał w Nowym Jorku wieżowiec Chryslera: przed I wojną światową czy po niej? ['po I wojnie światowej']
Która liczba jest większa: milion milionów czy tysiąc miliardów? ['są równe']
Wołyń leży na Litwie, Ukrainie czy Białorusi? ['na Ukrainie']
Który liczba jest większa: pierwiastek kwadratowy z 9 czy pierwiastek sześcienny z 27? ['są równe']
Który stopień jest wyższy: pułkownik czy komandor? ['są równe']
Premiera „Dziadów” w reżyserii Wyspiańskiego miała miejsce w XIX czy XX wieku? ['w XX', '20']
Film „Zemsta Sithów” opowiada o Rycerzach Okrągłego Stołu czy rycerzach Jedi? [

In [36]:
for q, answers in qa_groups[QuestionType.UNKNOWN]:
    if q.startswith("Któr") and any(answer.lower() in q.lower() for answer in answers):
        print(q, answers)

Które miasto jest stolicą Meksyku? ['Meksyk']
Który rozbiór Polski nastąpił w 1772 roku? ['pierwszy', '1']
Którą wielką literą alfabetu oznacza się stałą grawitacji Newtona? ['G']
Który wyraz w zdaniu „ten dom jest wysoki” jest przymiotnikiem? ['wysoki']
Którą literę w alfabecie Morse'a oznaczają trzy kropki? ['S']


# Experiments

### YES_NO

In [82]:
from collections import Counter

In [83]:
[(q, answers) for (q, answers) in qa_groups[QuestionType.YES_NO] if len(answers) > 1]

[('Czy ameba ma kształt?', ['nie', 'ma zmienny'])]

In [84]:
yes_no_counter = Counter([answers[0] for (q, answers) in qa_groups[QuestionType.YES_NO]])
yes_no_counter

Counter({'tak': 185, 'nie': 106, 'wokół głowy': 1})

In [85]:
yes_no_counter["tak"] / len(qa_groups[QuestionType.YES_NO])

0.6335616438356164

In [86]:
yes_no_counter["tak"] / len(questions_answers)

0.05285714285714286

In [293]:
(185 + 56) / len(questions_answers)

0.06885714285714285

### SELECT

In [88]:
import re

In [87]:
[(q, answers) for (q, answers) in qa_groups[QuestionType.SELECT] if len(answers) > 1]

[('Hakownica to broń sieczna czy palna?', ['palna', 'Londyn']),
 ('Utlenianie to przyjmowanie czy oddawanie elektronów?',
  ['oddawanie', 'Którym słowem określano damy dworskie?']),
 ('Markiza de Pompadour była faworytą Ludwika XV czy XVI?', ['XV', '15']),
 ('Premiera „Dziadów” w reżyserii Wyspiańskiego miała miejsce w XIX czy XX wieku?',
  ['w XX', '20'])]

In [112]:
len(qa_groups[QuestionType.SELECT])

231

In [113]:
pattern = re.compile("(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?")

In [171]:
matched = [qa for qa in qa_groups[QuestionType.SELECT] if pattern.match(qa[0])]
len(matched)

114

In [192]:
" ".join(opts[0].split()[-len(opts[-1].split()):])

'komórki glonów'

In [206]:
def tokenize(word):
    return filter(None, [
        re.search("\w*", t)[0]
        for t in word.lower().split()
    ])

In [207]:
import requests

In [208]:
SOLR_URL = "http://localhost:8983/solr/dialogs/query"

In [286]:
def select_counts(q):
    description, *opts = pattern.findall(q)[0]
    if len(opts[-1].split()) == 1:
        description = " ".join([description] + opts[0].split()[:-1])
        opts[0] = opts[0].split()[-1]
    counts = []
    description_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(description)])
    for opt in filter(None, opts):
        opt_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(opt)])
        solr_query = f'({description_query}) AND ({opt_query})'
        r = requests.get(
            SOLR_URL,
            json={"query": solr_query}
        )
        response = r.json()
        counts.append((response["response"]["numFound"], opt))
    counts = sorted(counts, reverse=True)
    return counts

In [290]:
def select2_counts(q):
    description, *opts = pattern.findall(q)[0]
    if len(opts[-1].split()) == 1:
        description = " ".join([description] + opts[0].split()[:-1])
        opts[0] = opts[0].split()[-1]
    counts = []
    description_query = " OR ".join([f"title_txt_pl:{t}" for t in tokenize(description)])
    for opt in filter(None, opts):
        opt_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(opt)])
        solr_query = f'({description_query}) AND ({opt_query})'
        r = requests.get(
            SOLR_URL,
            json={"query": solr_query}
        )
        response = r.json()
        counts.append((response["response"]["numFound"], opt))
    counts = sorted(counts, reverse=True)
    return counts

In [289]:
q = "Tuja to roślina iglasta czy liściasta?"
description, *opts = pattern.findall(q)[0]
if len(opts[-1].split()) == 1:
    description = " ".join([description] + opts[0].split()[:-1])
    opts[0] = opts[0].split()[-1]
counts = []
description_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(description)])
for opt in filter(None, opts):
    opt_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(opt)])
    solr_query = f'({description_query}) AND ({opt_query})'
    r = requests.get(
        SOLR_URL,
        json={"query": solr_query}
    )
    response = r.json()
    counts.append((response["response"]["numFound"], opt))
counts = sorted(counts, reverse=True)
print(solr_query)
print(description)

(content_txt_pl:tuja OR content_txt_pl:roślina) AND (content_txt_pl:liściasta)
Tuja roślina


In [277]:
from tqdm import tqdm

In [ ]:
q_counts = [
    select2_counts(q)
    for (q, _) in tqdm(matched)
]

In [292]:
answer_match = 0
for (q, answers), counts in zip(matched, q_counts):
    # counts = select_counts(q)
    a = counts[0][1]
    if match(a, answers):
        answer_match += 1
    else:
        print(q, answers, a, counts)
answer_match

Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia? ['punkt zero'] linia odcięcia [(2365, 'linia odcięcia'), (2012, 'punkt zero')]
Baron to tytuł wyższy czy niższy niż hrabia? ['niższy'] tytuł wyższy [(69, 'tytuł wyższy'), (26, 'niższy niż hrabia')]
Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich? ['zbiór starożytnych rzeźb greckich'] świątynia w Atenach [(7, 'świątynia w Atenach'), (2, 'zbiór starożytnych rzeźb greckich')]
Średniowieczne gęśle to instrument szarpany czy dęty? ['szarpany'] dęty [(15, 'dęty'), (9, 'szarpany')]
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem'] po ślubie [(4, 'po ślubie'), (4, 'okres przed')]
Dybuk to dusza zmarłego grzesznika wstępująca do ciała żywego czy duch w domostwie domowym? ['dusza zmarłego grzesznika wstępująca do ciała żywego'] duch w domostwie domowym [(4, 'duch w domostwie domowym'), (2, 'dusza zmarłego grzesznika wstępująca do ciała żywego')]
Diablotka to ostro przyprawiona

63

In [ ]:
q_counts = [
    select_counts(q)
    for (q, _) in tqdm(matched)
]

In [288]:
answer_match = 0
for (q, answers), counts in zip(matched, q_counts):
    # counts = select_counts(q)
    a = counts[0][1]
    if match(a, answers):
        answer_match += 1
    else:
        print(q, answers, a, counts)
answer_match

Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia? ['punkt zero'] linia odcięcia [(1070196, 'linia odcięcia'), (1069161, 'punkt zero')]
Baron to tytuł wyższy czy niższy niż hrabia? ['niższy'] tytuł wyższy [(89197, 'tytuł wyższy'), (39389, 'niższy niż hrabia')]
Czy sombrero to kapelusz, danie czy taniec? ['kapelusz'] danie [(84143, 'danie'), (2122, 'taniec'), (525, 'kapelusz')]
Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich? ['zbiór starożytnych rzeźb greckich'] świątynia w Atenach [(1067149, 'świątynia w Atenach'), (32315, 'zbiór starożytnych rzeźb greckich')]
Średniowieczne gęśle to instrument szarpany czy dęty? ['szarpany'] dęty [(7512, 'dęty'), (7403, 'szarpany')]
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem'] okres przed [(53915, 'okres przed'), (14838, 'po ślubie')]
Świekier to dawna polska nazwa ojca żony czy ojca męża? ['ojca męża'] dawna polska nazwa ojca żony [(386283, 'dawna polska nazwa ojca żony'), (2

49

In [283]:
answer_match = 0
for (q, answers), counts in zip(matched, q_counts):
    # counts = select_counts(q)
    a = counts[0][1]
    if match(a, answers):
        answer_match += 1
    else:
        print(q, answers, a, counts)
answer_match

Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia? ['punkt zero'] linia odcięcia [(31263, 'linia odcięcia'), (25492, 'punkt zero')]
Baron to tytuł wyższy czy niższy niż hrabia? ['niższy'] tytuł wyższy [(558, 'tytuł wyższy'), (449, 'niższy niż hrabia')]
Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny? ['sprytny'] naiwny [(173, 'naiwny'), (126, 'sprytny')]
Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich? ['zbiór starożytnych rzeźb greckich'] świątynia w Atenach [(588, 'świątynia w Atenach'), (152, 'zbiór starożytnych rzeźb greckich')]
Średniowieczne gęśle to instrument szarpany czy dęty? ['szarpany'] dęty [(238, 'dęty'), (83, 'szarpany')]
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem'] okres przed [(1122, 'okres przed'), (160, 'po ślubie')]
Dybuk to dusza zmarłego grzesznika wstępująca do ciała żywego czy duch w domostwie domowym? ['dusza zmarłego grzesznika wstępująca do ciała żywego'] duch w domostwie domowym [

56

In [268]:
answer_match = 0
for (q, answers), counts in zip(matched, q_counts):
    # counts = select_counts(q)
    a = counts[0][1]
    if match(a, answers):
        answer_match += 1
    else:
        print(q, answers, a, counts)
answer_match

Baron to tytuł wyższy czy niższy niż hrabia? ['niższy'] tytuł wyższy [(0, 'tytuł wyższy'), (0, 'niższy niż hrabia')]
Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny? ['sprytny'] naiwny [(173, 'naiwny'), (126, 'sprytny')]
Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich? ['zbiór starożytnych rzeźb greckich'] świątynia w Atenach [(0, 'świątynia w Atenach'), (0, 'zbiór starożytnych rzeźb greckich')]
Średniowieczne gęśle to instrument szarpany czy dęty? ['szarpany'] dęty [(238, 'dęty'), (83, 'szarpany')]
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem'] okres przed [(1122, 'okres przed'), (160, 'po ślubie')]
Diablotka to ostro przyprawiona zupa rybna czy placek z ciasta francuskiego? ['placek'] placek z ciasta francuskiego [(0, 'placek z ciasta francuskiego'), (0, 'ostro przyprawiona zupa rybna')]
Ju-jitsu to starojapoński styl walki wręcz czy przy pomocy kijów bambusowych? ['wręcz'] starojapoński styl walki wręcz [(0, 'starojapoński styl walki w

56

In [253]:
for counts, qa in zip(q_counts, matched):
    print(counts, qa)

[(5, 'punkt zero'), (1, 'linia odcięcia')] ('Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia?', ['punkt zero'])
[(0, 'tytuł wyższy'), (0, 'niższy niż hrabia')] ('Baron to tytuł wyższy czy niższy niż hrabia?', ['niższy'])
[(173, 'naiwny'), (126, 'ktoś sprytny')] ('Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny?', ['sprytny'])
[(3, 'kapelusz'), (1, 'taniec'), (1, 'danie')] ('Czy sombrero to kapelusz, danie czy taniec?', ['kapelusz'])
[(0, 'świątynia w Atenach'), (0, 'zbiór starożytnych rzeźb greckich')] ('Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich?', ['zbiór starożytnych rzeźb greckich'])
[(21, 'związek chemiczny'), (4, 'pierwiastek')] ('Wapno palone to związek chemiczny czy pierwiastek?', ['związek chemiczny'])
[(3, 'dęty'), (0, 'instrument szarpany')] ('Średniowieczne gęśle to instrument szarpany czy dęty?', ['szarpany'])
[(1122, 'okres przed'), (160, 'po ślubie')] ('Miesiąc miodowy to okres przed czy po ślubie?', ['p

In [251]:
answer_match = 0
for q, answers in tqdm(matched):
    counts = select_counts(q)
    a = counts[0][1]
    if match(a, answers):
        answer_match += 1
answer_match

100%|██████████| 114/114 [00:15<00:00,  7.44it/s]


49

In [246]:
answer_match = 0
for q, answers in tqdm(matched):
    counts = select_counts(q)
    a = counts[0][1]
    if match(a, answers):
        answer_match += 1
answer_match

100%|██████████| 114/114 [00:11<00:00,  9.50it/s]


51

In [238]:
q_counts = [
    select_counts(q)
    for (q, _) in tqdm(matched)
]

100%|██████████| 114/114 [00:11<00:00, 10.02it/s]


In [241]:
q_counts[0]

[(31263, 'linia odcięcia'), (25492, 'punkt zero')]

In [243]:
answer_match = 0
for q, answers in tqdm(matched):
    a = response_select(q)
    if match(a, answers):
        answer_match += 1
answer_match

100%|██████████| 114/114 [00:30<00:00,  3.70it/s]


47

In [223]:
answer_match = 0
for q, answers in tqdm(matched):
    description, *opts = pattern.findall(q)[0]
    if len(opts[-1].split()) == 1:
        opts[0] = opts[0].split()[-1]
    a = opts[0]
    if match(a, answers):
        answer_match += 1
answer_match

100%|██████████| 114/114 [00:00<00:00, 14309.89it/s]


61

In [224]:
answer_match / len(questions_answers)

0.01742857142857143

In [216]:

match(response_select(q), answers)

True

In [204]:
answer_match = 0
first_match = 0
for q, answers in matched:
    description, *opts = pattern.findall(q)[0]
    if len(opts[-1].split()) == 1:
        opts[0] = opts[0].split()[-1]
    # elif len(opts[0].split()) == 1:
    #     opts[-1] = opts[-1].split()[0]
    # opts[0] = " ".join(opts[0].split()[-len(opts[-1].split()):])
    if match(opts[0], answers):
        first_match += 1
    for opt in filter(None, opts): # remove empty
        if match(opt, answers):
            # print(opt, answers)
            answer_match += 1
            break
    else:
        print(q, answers, opts)
answer_match, first_match

Baron to tytuł wyższy czy niższy niż hrabia? ['niższy'] ['tytuł wyższy', '', 'niższy niż hrabia']
Miesiąc miodowy to okres przed czy po ślubie? ['przed ślubem'] ['okres przed', '', 'po ślubie']
Diablotka to ostro przyprawiona zupa rybna czy placek z ciasta francuskiego? ['placek'] ['ostro przyprawiona zupa rybna', '', 'placek z ciasta francuskiego']
Ju-jitsu to starojapoński styl walki wręcz czy przy pomocy kijów bambusowych? ['wręcz'] ['starojapoński styl walki wręcz', '', 'przy pomocy kijów bambusowych']
Kirys to nazwa broni siecznej, palnej czy części zbroi rycerskiej? ['części zbroi'] ['nazwa broni siecznej', 'palnej', 'części zbroi rycerskiej']
Utlenianie to przyjmowanie czy oddawanie elektronów? ['oddawanie', 'Którym słowem określano damy dworskie?'] ['przyjmowanie', '', 'oddawanie elektronów']
Konformista to człowiek bezkrytycznie podporządkowujący się normom czy też sprzeciwiający się im? ['podporządkowujący się normom'] ['człowiek bezkrytycznie podporządkowujący się normom', '

(96, 61)

In [164]:

description, *opts = pattern.findall([q for (q, _) in qa_groups[QuestionType.SELECT] if pattern.match(q)][10])[0]

In [ ]:
def inspect(q):
    

In [294]:
qa_groups[QuestionType.NAME]

[('Jak nazywa się pierwsza litera alfabetu greckiego?', ['alfa']),
 ('Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?', ['cięciwa']),
 ('Jak nazywa się pierwiastek o symbolu Bq?', ['becquerel']),
 ('Jak nazywał się sztywny kapelusz męski o zaokrąglonej główce i wąskim lekko uniesionym rondelku, modny w drugiej połowie XIX wieku?',
  ['melonik']),
 ('Jak nazywa się wypukła albo wklęsła powierzchnia cieczy w pobliżu ścianek naczynia?',
  ['menisk']),
 ('Jak nazywał się fizyk, który w 1876 r. wynalazł telefon?',
  ['Aleksander Graham Bell']),
 ('Jak nazywa się wzgórze w Paryżu na którym wybudowano bazylikę Sacré-Cœur?',
  ['Montmartre']),
 ('Jak nazywa się trucizna przyrządzona z korzenia szaleju?', ['cykuta']),
 ('Jak nazywa się stolica Kuby?', ['Hawana']),
 ('Jak nazywa się agent brytyjskiego wywiadu, bohater powieści Iana Fleminga?',
  ['James Bond']),
 ('Jak nazywa się przyrząd do palenia i górny kieł dzika?', ['fajka']),
 ('Jak nazywa się potomek osadników portugalskich bąd

In [165]:
description

'Dybuk'

In [167]:
opts

['dusza zmarłego grzesznika wstępująca do ciała żywego',
 '',
 'duch w domostwie domowym']

In [143]:
pattern2 = re.compile(".* czy (.*?)\?$")

In [205]:
[q for (q, _) in qa_groups[QuestionType.SELECT] if not pattern.match(q)]

['Według przysłowia nos jest dla tabakiery czy tabakiera dla nosa?',
 'Który zwrotnik przecina Meksyk – Raka czy Koziorożca?',
 'Wyspy Zielonego Przylądka leżą bliżej Afryki czy Azji?',
 'Czy utwór „Mikołaja Doświadczyńskiego przypadki” Ignacego Krasickiego napisany jest wierszem czy prozą?',
 'Sahara Zachodnia posiada duże złoża miedzi, fosforytów czy uranu?',
 'Co ile lat odradzał się z popiołów mityczny feniks – co 50, co 500 czy co 5000 lat?',
 'Kto wcześniej był koronowany na króla Polski: Jagiełło czy Jadwiga?',
 'Żeby odwiedzić miasto Azteków, trzeba pojechać do Meksyku, Boliwii czy Peru?',
 'W której epoce żył św. Tomasz z Akwinu: w średniowieczu czy renesansie?',
 'Która liczba jest większa: -1 czy -1,5?',
 'Mianem X muzy określa się telewizję czy film?',
 'Czy ptak o nazwie sekretarz jest drapieżny czy roślinożerny?',
 'Powierzchnia boiska do badmintona jest większa czy mniejsza od boiska do tenisa?',
 'John Glenn czy Walentyna Tierieszkowa – kto wcześniej był w kosmosie?',
 

In [146]:
[q for (q, _) in qa_groups[QuestionType.SELECT] if not pattern.match(q) and not pattern2.match(q)]

['Laudację można wybić czy wygłosić']

In [126]:
from spacy import displacy

In [125]:
list(a.sents)

[Który zwrotnik przecina Meksyk – Raka czy Koziorożca?]

In [115]:
len([q for (q, _) in qa_groups[QuestionType.SELECT] if pattern.match(q)])

114

In [105]:
q, answers = qa_groups[QuestionType.SELECT][1]
q, answers

('Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia?',
 ['punkt zero'])

In [102]:
match("tabakiera dla nosa".lower(), map(str.lower, answers))

True

<re.Match object; span=(0, 79), match='Punkt przecięcia osi w układzie współrzędnych to >

In [108]:
re.match(pattern, q)

<re.Match object; span=(0, 79), match='Punkt przecięcia osi w układzie współrzędnych to >

In [106]:
re.findall(pattern, q)

[('Punkt przecięcia osi w układzie współrzędnych',
  'punkt zero',
  '',
  'linia odcięcia')]

In [13]:
from typing import Dict, Tuple

import pandas as pd
import requests

SOLR_URL = "http://localhost:8983/solr/dialogs/query"


def query_solr(query: str, language: str) -> Tuple[str, Dict]:
    """
    Function to query the Solr database with a question

    Parameters
    ----------
        query, str:
            The question to be searched
        language, str:
            Question language abbreviation
    """
    if query is None:
        return None, None

    solr_query = f"content_txt_{language}: {query} OR "
    solr_query += " OR ".join([f"content_txt_{language}:{t}" for t in query.split()])

    r = requests.get(
        SOLR_URL,
        json={"query": solr_query, "params": {"debugQuery": True}}
    )

    if not r.ok:
        return None, r.text
    response = r.json()
    docs = response["response"]["docs"]
    if not docs:
        return None, response
    info = {
        "solr_query": solr_query,
        "response": response,
    }
    return docs[0][f"title_txt_{language}"], info

In [14]:
Q = "Wapno palone to związek chemiczny czy pierwiastek?"

In [15]:
for description, *opts in re.findall("(.*?) to (.*?) czy (.*?)\?", Q):
    counts = []
    for opt in opts:
        q = description + " to " + opt
        solr_query = f'content_txt_pl:"{description}" AND content_txt_pl:"{opt}"'
        r = requests.get(
            SOLR_URL,
            json={"query": solr_query, "params": {"debugQuery": False}}
        )
        response = r.json()
        docs = response["response"]["docs"]
        counts.append((len(docs), opt))

In [16]:
sorted(counts)

[(0, 'pierwiastek'), (5, 'związek chemiczny')]

In [18]:
from tqdm import tqdm

In [27]:
a.split("\n")[0]

'Jak nazywa się pierwsza litera alfabetu greckiego?\talfa'

In [50]:
re.findall("^(.*?) to (.*?) czy (.*?)\?\s*(.*?)$", 'Wapno palone to związek chemiczny czy pierwiastek?\tzwiązek chemiczny')

[('Wapno palone', 'związek chemiczny', 'pierwiastek', 'związek chemiczny')]

In [105]:
with open("data/questions.tsv", "rt") as f:
    a = f.read()

In [72]:
re.findall("^(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?\s*(.*?)$", "Oryks to antylopa, pies czy kot?	antylopa")

[('Oryks', 'antylopa', 'pies', 'kot', 'antylopa')]

In [70]:
solr_query

'content_txt_pl:punkt OR content_txt_pl:przecięcia OR content_txt_pl:osi OR content_txt_pl:w OR content_txt_pl:układzie OR content_txt_pl:współrzędnych AND '

In [100]:
re.findall("(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?", "Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny?")

[('Ktoś „nie w ciemię bity”', 'ktoś sprytny', '', 'naiwny')]

In [85]:
re.search("\w*", "abc")[0]

'abc'

In [91]:
def tokenize(word):
    return filter(None, [
        re.search("\w*", t)[0]
        for t in word.lower().split()
    ])

In [98]:
response

{'responseHeader': {'status': 0,
  'QTime': 0,
  'params': {'json': '{"query": "content_txt_pl:diablotka AND content_txt_pl:ostro OR content_txt_pl:przyprawiona OR content_txt_pl:zupa OR content_txt_pl:rybna", "params": {"debugQuery": false}}'}},
 'response': {'numFound': 0, 'start': 0, 'numFoundExact': True, 'docs': []}}

In [107]:
for l in re.finditer("^(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?\s*(.*?)$", a, flags=re.MULTILINE):
    print(l[0])

Punkt przecięcia osi w układzie współrzędnych to punkt zero czy linia odcięcia?	punkt zero
Baron to tytuł wyższy czy niższy niż hrabia?	niższy
Ktoś „nie w ciemię bity” to ktoś sprytny czy naiwny?	sprytny
Czy sombrero to kapelusz, danie czy taniec?	kapelusz
Marmury Elgina to świątynia w Atenach czy zbiór starożytnych rzeźb greckich?	zbiór starożytnych rzeźb greckich
Wapno palone to związek chemiczny czy pierwiastek?	związek chemiczny
Średniowieczne gęśle to instrument szarpany czy dęty?	szarpany
Miesiąc miodowy to okres przed czy po ślubie?	przed ślubem
Hakownica to broń sieczna czy palna?	palna	Londyn
Świekier to dawna polska nazwa ojca żony czy ojca męża?	ojca męża
Dybuk to dusza zmarłego grzesznika wstępująca do ciała żywego czy duch w domostwie domowym?	dusza zmarłego grzesznika wstępująca do ciała żywego
Hakownica to sprzęt alpejski czy broń?	broń
Diablotka to ostro przyprawiona zupa rybna czy placek z ciasta francuskiego?	placek
Synogarlice to odmiana gołębi czy sikorek?	gołębi
Ju

In [104]:
a

<re.Match object; span=(241130, 241194), match='Urwistki to komórki glonów czy odnóża świerszczy?>

In [99]:
for description, *opts, correct in re.findall("^(.*?) to (.*?)(?:, (.*?))* czy (.*?)\?\s*(.*?)$", a, flags=re.MULTILINE):
    counts = []
    description_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(description)])
    for opt in filter(None, opts):
        opt_query = " OR ".join([f"content_txt_pl:{t}" for t in tokenize(opt)])
        solr_query = f"{description_query} AND {opt_query}"
        r = requests.get(
            SOLR_URL,
            json={"query": solr_query}
        )
        response = r.json()
        counts.append((response["response"]["numFound"], opt))
    print(description + " to " + correct, list(reversed(sorted(counts))))

Punkt przecięcia osi w układzie współrzędnych to punkt zero [(247, 'punkt zero'), (66, 'linia odcięcia')]
Baron to niższy [(495, 'tytuł wyższy'), (29, 'niższy niż hrabia')]
Ktoś „nie w ciemię bity” to sprytny [(1931, 'ktoś sprytny'), (1, 'naiwny')]
Czy sombrero to kapelusz [(3, 'kapelusz'), (1, 'taniec'), (1, 'danie')]
Marmury Elgina to zbiór starożytnych rzeźb greckich [(1, 'świątynia w Atenach'), (0, 'zbiór starożytnych rzeźb greckich')]
Wapno palone to związek chemiczny [(67, 'związek chemiczny'), (3, 'pierwiastek')]
Średniowieczne gęśle to szarpany [(10, 'instrument szarpany'), (0, 'dęty')]
Miesiąc miodowy to przed ślubem [(240, 'po ślubie'), (23, 'okres przed')]
Hakownica to palna	Londyn [(10, 'broń sieczna'), (8, 'palna')]
Świekier to ojca męża [(0, 'ojca męża'), (0, 'dawna polska nazwa ojca żony')]
Dybuk to dusza zmarłego grzesznika wstępująca do ciała żywego [(2, 'dusza zmarłego grzesznika wstępująca do ciała żywego'), (1, 'duch w domostwie domowym')]
Hakownica to broń [(10, 'b